##### Copyright 2020 The OpenFermion Developers

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hamiltonian Time Evolution and Expectation Value Computation

This tutorial describes the FQE's capabilities for Hamiltonian time-evolution and expectation value estimation

Where possible, LiH will be used as an example molecule for the API.

In [1]:
try:
    import fqe
except ImportError:
    !pip install fqe --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 597.5/597.5 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
Print = True
from openfermion import FermionOperator, MolecularData
from openfermion.utils import hermitian_conjugated
import numpy
import fqe

numpy.set_printoptions(floatmode='fixed', precision=6, linewidth=80, suppress=True)
numpy.random.seed(seed=409)

In [3]:
!curl -O https://raw.githubusercontent.com/quantumlib/OpenFermion-FQE/master/tests/unittest_data/build_lih_data.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 62086  100 62086    0     0   174k      0 --:--:-- --:--:-- --:--:--  174k


In [4]:
import build_lih_data

In [6]:
h1e, h2e, wfn = build_lih_data.build_lih_data('energy')
lih_hamiltonian = fqe.get_restricted_hamiltonian(([h1e, h2e]))
lihwfn = fqe.Wavefunction([[4, 0, 6]])
lihwfn.set_wfn(strategy='from_data', raw_data={(4, 0): wfn})
if Print:
    lihwfn.print_wfn()

Sector N = 4 : S_z = 0
a'000011'b'000011' (-0.9870890035778126+0j)
a'000011'b'000101' (-0.0384854879340124+0j)
a'000011'b'100001' (-0.0036477466809967+0j)
a'000101'b'000011' (-0.0384854879340073+0j)
a'000101'b'000101' (0.0344508540105218+0j)
a'000101'b'100001' (-0.0592953695921639+0j)
a'000101'b'100010' (-0.0010777697106179+0j)
a'001001'b'001001' (0.0271656962509733+0j)
a'001001'b'001010' (-0.0031209939957822+0j)
a'010001'b'010001' (0.0271656962509733+0j)
a'010001'b'010010' (-0.0031209939957822+0j)
a'100001'b'000011' (-0.003647746680997+0j)
a'100001'b'000101' (-0.0592953695921649+0j)
a'100001'b'100001' (0.1135892009408387+0j)
a'000110'b'000110' (0.0037623492033852+0j)
a'001010'b'001001' (-0.0031209939957822+0j)
a'001010'b'001010' (0.001794202149606+0j)
a'010010'b'010001' (-0.0031209939957822+0j)
a'010010'b'010010' (0.001794202149606+0j)
a'100010'b'000101' (-0.001077769710618+0j)
a'100010'b'100010' (0.0010515758855392+0j)


## Application of one- and two-body fermionic gates

The API for time propagation can be invoked through the fqe namespace or the wavefunction object

In [7]:
# dummy geometry
from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion import jordan_wigner, get_sparse_operator, InteractionOperator, get_fermion_operator

h1s, h2s = spinorb_from_spatial(h1e, numpy.einsum("ijlk", -2 * h2e) * 0.5)
mol = InteractionOperator(0, h1s, h2s)
ham_fop = get_fermion_operator(mol)
ham_mat = get_sparse_operator(jordan_wigner(ham_fop)).toarray()


In [ ]:
from scipy.linalg import expm
time = 0.01
evolved1 = lihwfn.time_evolve(time, lih_hamiltonian)
if Print:
    evolved1.print_wfn()
evolved2 = fqe.time_evolve(lihwfn, time, lih_hamiltonian)
if Print:
    evolved2.print_wfn()
assert numpy.isclose(fqe.vdot(evolved1, evolved2), 1)
cirq_wf = fqe.to_cirq(lihwfn)
evolve_cirq = expm(-1j * time * ham_mat) @ cirq_wf
test_evolve = fqe.from_cirq(evolve_cirq, thresh=1.0E-12)
assert numpy.isclose(fqe.vdot(test_evolve, evolved1), 1)

In [9]:
import numpy as np # Use np alias for consistency
import fqe
from openfermion import jordan_wigner, get_sparse_operator, InteractionOperator, get_fermion_operator
from openfermion.chem.molecular_data import spinorb_from_spatial
from openfermion import FermionOperator # For building the number operator later

# --- Building the OpenFermion Hamiltonian matrix ---
# Number of spatial orbitals from the input h1e integral dimensions
n_spatial_orbitals_from_h1e = h1e.shape[0]
n_spin_orbitals_for_ham = 2 * n_spatial_orbitals_from_h1e # Total qubits for the Hamiltonian

h1s, h2s = spinorb_from_spatial(h1e, np.einsum("ijlk", -2 * h2e) * 0.5)
mol = InteractionOperator(0, h1s, h2s)
ham_fop = get_fermion_operator(mol)
ham_mat = get_sparse_operator(jordan_wigner(ham_fop), n_qubits=n_spin_orbitals_for_ham).toarray() # Pass n_qubits explicitly

print(f"\n--- Hamiltonian Matrix Details ---")
print(f"Number of spin-orbitals in Hamiltonian: {n_spin_orbitals_for_ham}")
print(f"Dimension of Hamiltonian matrix: {ham_mat.shape[0]}x{ham_mat.shape[1]}")


# --- Get the True Ground State Energy and Eigenvector by Diagonalization ---
print(f"\n--- Performing Exact Diagonalization ---")

# numpy.linalg.eigh is used for Hermitian matrices (like quantum Hamiltonians)
# It returns eigenvalues and eigenvectors. Eigenvalues are typically sorted in ascending order.
eigenvalues, eigenvectors = np.linalg.eigh(ham_mat)

# The first eigenvalue (index 0) is the ground state energy
ground_state_energy_true = eigenvalues[0]

# The corresponding eigenvector is the first column of the eigenvectors matrix
ground_state_eigenvector_true = eigenvectors[:, 0]

print(f"True Ground State Energy (Eigenvalue): {ground_state_energy_true:.6f} Ha")
print(f"Ground State Eigenvector (first 10 elements):")
print(ground_state_eigenvector_true[:10]) # Print first few elements as it can be very long
print(f"Norm of Ground State Eigenvector: {np.linalg.norm(ground_state_eigenvector_true):.6f}")

# --- Optional: Verify Electron Number of the True Ground State Eigenvector ---
# For the LiH example, we expect 4 electrons.
expected_num_electrons = 4 # Based on your fqe.Wavefunction([[4, 0, 6]]) setup

# Build the number operator sum_p a_p^dagger a_p
n_op_fermion = FermionOperator()
for p in range(n_spin_orbitals_for_ham):
    n_op_fermion += FermionOperator(((p, 1), (p, 0))) # a_p_dagger a_p

n_op_qubit = jordan_wigner(n_op_fermion)
n_op_sparse = get_sparse_operator(n_op_qubit, n_qubits=n_spin_orbitals_for_ham)

# Calculate expectation value of number operator with the true ground state eigenvector
n_electrons_in_true_gs = np.vdot(ground_state_eigenvector_true, n_op_sparse @ ground_state_eigenvector_true).real

print(f"Number of electrons in true ground state eigenvector: {n_electrons_in_true_gs:.0f}")
print(f"(Expected: {expected_num_electrons} electrons)")

# # --- Comparison with FQE wavefunction's energy (from previous step) ---
# # Calculate the expectation value of the Hamiltonian with the FQE wavefunction loaded earlier
# energy_of_fqe_wfn = lihwfn.expectation_value(lih_hamiltonian)
# print(f"\nEnergy of the provided FQE wavefunction (lihwfn): {energy_of_fqe_wfn:.6f} Ha")

print("\n--- Summary ---")
print("1. The 'Energy of the provided FQE wavefunction' is the energy of the specific wavefunction")
print("   'lihwfn' that you set from your 'wfn_data'.")
print("2. The 'True Ground State Energy' is the absolute minimum energy eigenvalue of the")
print("   Hamiltonian matrix 'ham_mat', obtained via exact diagonalization. Its corresponding")
print("   vector is the 'ground_state_eigenvector_true'.")
print("If 'wfn_data' represents the exact ground state (e.g., from an FCI calculation),")
print("these two energy values should be very close.")


--- Hamiltonian Matrix Details ---
Number of spin-orbitals in Hamiltonian: 12
Dimension of Hamiltonian matrix: 4096x4096

--- Performing Exact Diagonalization ---
True Ground State Energy (Eigenvalue): -8.877720 Ha
Ground State Eigenvector (first 10 elements):
[-0.000000+0.000000j  0.000000+0.000000j -0.000000+0.000000j
  0.000000+0.000000j -0.000000+0.000000j -0.000000+0.000000j
 -0.000000+0.000000j  0.000000+0.000000j  0.000000+0.000000j
 -0.000000+0.000000j]
Norm of Ground State Eigenvector: 1.000000
Number of electrons in true ground state eigenvector: 4
(Expected: 4 electrons)


AttributeError: 'Wavefunction' object has no attribute 'expectation_value'

In [15]:
eigenvectors

array([[-0.000000+0.000000j,  0.000000+0.000000j,  0.000000+0.000000j, ...,
         1.000000+0.000000j, -0.000000+0.000000j,  0.000000+0.000000j],
       [ 0.000000+0.000000j, -0.000000+0.000000j,  0.000000+0.000000j, ...,
        -0.000000+0.000000j,  0.000000+0.000000j,  0.000000+0.000000j],
       [-0.000000+0.000000j,  0.000000+0.000000j,  0.000000+0.000000j, ...,
         0.000000+0.000000j,  0.000000+0.000000j,  0.000000+0.000000j],
       ...,
       [-0.000000+0.000000j, -0.000000+0.000000j,  0.000000+0.000000j, ...,
         0.000000+0.000000j, -0.000000+0.000000j,  0.000000+0.000000j],
       [ 0.000000+0.000000j,  0.000000+0.000000j, -0.000000+0.000000j, ...,
        -0.000000+0.000000j, -0.000000+0.000000j, -0.000000+0.000000j],
       [-0.000000+0.000000j, -0.000000+0.000000j, -0.000000+0.000000j, ...,
         0.000000+0.000000j,  1.000000+0.000000j,  0.000000+0.000000j]])

In [ ]:
import numpy as np

def analyze_eigenvector_contributions(eigvec, threshold=1e-10):
    """
    Analyzes an eigenvector, classifying its components as significant or
    effectively zero, and mapping them to fermionic Fock states in a tabular format.

    Args:
        eigvec (np.ndarray): The eigenvector array.
        threshold (float): The absolute amplitude value below which a component
                           is considered "effectively zero" (numerical noise).
    """
    if not isinstance(eigvec, np.ndarray) or eigvec.ndim != 1:
        print("Error: eigvec must be a 1D NumPy array.")
        return

    num_components = len(eigvec)
    if num_components == 0:
        print("Error: eigvec is empty.")
        return

    # Determine n_spin_orbitals from the length of the eigenvector
    # It must be a power of 2 for a valid qubit/fermionic basis
    n_spin_orbitals = int(np.log2(num_components))
    if 2**n_spin_orbitals != num_components:
        print(f"Error: Eigenvector length ({num_components}) is not a power of 2, "
              "which is required for this basis interpretation.")
        return

    print("--- Detailed Analysis of Each Basis State Contribution (Tabular Format) ---")
    # print("Spin Orbital Mapping (Jordan-Wigner convention, Qubit i <-> Orbital i):")
    # print("  Orbital 0: Site 0, Spin-Up")
    # print("  Orbital 1: Site 0, Spin-Down")
    # print("  Orbital 2: Site 1, Spin-Up")
    # print("  Orbital 3: Site 1, Spin-Down")
    # print(f"Basis states are ordered as |Orb{n_spin_orbitals-1} ... Orb1 Orb0⟩ (or |Qubit{n_spin_orbitals-1} ... Qubit1 Qubit0⟩).\n")

    # Prepare data for the table
    table_data = []

    for i, amplitude in enumerate(eigvec):
        is_significant = abs(amplitude) > threshold
        classification = "Significant" if is_significant else "Effectively 0"

        binary_state = bin(i)[2:].zfill(n_spin_orbitals)
        occupied_orbitals = [k for k, bit_char in enumerate(reversed(binary_state)) if bit_char == '1']
        num_electrons = len(occupied_orbitals)

        description_parts = []
        if 0 in occupied_orbitals: description_parts.append("Site0_Up")
        if 1 in occupied_orbitals: description_parts.append("Site0_Down")
        if 2 in occupied_orbitals: description_parts.append("Site1_Up")
        if 3 in occupied_orbitals: description_parts.append("Site1_Down")

        orbital_description = "Vacuum (0 electrons)" if not description_parts else f"{num_electrons} electrons: {', '.join(description_parts)}"

        # Format amplitude string for display
        if not is_significant and np.isclose(amplitude, 0j):
            amplitude_str = "0.0000+0.0000j"
        else:
            amplitude_str = f"{amplitude.real:.4f}{'+' if amplitude.imag >= 0 else ''}{amplitude.imag:.4f}j"

        table_data.append({
            'index': i,
            'amplitude': amplitude_str,
            'classification': classification,
            'binary_state': binary_state,
            'num_electrons': num_electrons,
            'occupied_orbitals': str(occupied_orbitals), # Convert list to string for printing
            'description': orbital_description
        })

    # Determine column widths for formatting
    col_widths = {
        'index': 5,
        'amplitude': 20,
        'classification': 15,
        'binary_state': 12,
        'num_electrons': 7,
        'occupied_orbitals': 20,
        'description': 40
    }

    # Print header
    header = (
        f"{'Index':<{col_widths['index']}} | "
         f"{'|Basis State>':<{col_widths['binary_state']}} | "
        f"{'Amplitude':<{col_widths['amplitude']}} | "
        f"{'Type':<{col_widths['classification']}} | "
        f"{'#Elec.':<{col_widths['num_electrons']}} | "
        f"{'Occ. Orbs':<{col_widths['occupied_orbitals']}} | "
        f"{'Description':<{col_widths['description']}}"
    )
    print(header)
    print("-" * len(header))

    # Print data rows
    for row in table_data:
        print(
            f"{row['index']:<{col_widths['index']}} | "
                        f"{'|' + row['binary_state'] + '⟩':<{col_widths['binary_state']}} | " # Add | and ⟩ here

            f"{row['amplitude']:<{col_widths['amplitude']}} | "
            f"{row['classification']:<{col_widths['classification']}} | "
            f"{row['num_electrons']:<{col_widths['num_electrons']}} | "
            f"{row['occupied_orbitals']:<{col_widths['occupied_orbitals']}} | "
            f"{row['description']:<{col_widths['description']}}"
        )

    print("\n---------------------------------------------------")
    total_abs_sq_amplitude = np.sum(np.abs(eigvec)**2)
    print(f"Sum of squares of absolute amplitudes (should be ~1.0): {total_abs_sq_amplitude:.6f}")
    if np.isclose(total_abs_sq_amplitude, 1.0):
        print("Eigenvector is normalized (within numerical precision).")
    else:
        print("WARNING: Eigenvector is NOT normalized.")

# --- Example Usage ---


# Call the function with your eigenvector
analyze_eigenvector_contributions(eigenvectors.flatten())

--- Detailed Analysis of Each Basis State Contribution (Tabular Format) ---
Spin Orbital Mapping (Jordan-Wigner convention, Qubit i <-> Orbital i):
  Orbital 0: Site 0, Spin-Up
  Orbital 1: Site 0, Spin-Down
  Orbital 2: Site 1, Spin-Up
  Orbital 3: Site 1, Spin-Down
Basis states are ordered as |Orb23 ... Orb1 Orb0⟩ (or |Qubit23 ... Qubit1 Qubit0⟩).



## Exact evolution implementation of quadratic Hamiltonians

Listed here are examples of evolving the special Hamiltonians.

Diagonal Hamiltonian evolution is supported.

In [ ]:
wfn = fqe.Wavefunction([[4, 2, 4]])
wfn.set_wfn(strategy='random')
if Print:
    wfn.print_wfn()

diagonal = FermionOperator('0^ 0', -2.0) + \
           FermionOperator('1^ 1', -1.7) + \
           FermionOperator('2^ 2', -0.7) + \
           FermionOperator('3^ 3', -0.55) + \
           FermionOperator('4^ 4', -0.1) + \
           FermionOperator('5^ 5', -0.06) + \
           FermionOperator('6^ 6', 0.5) + \
           FermionOperator('7^ 7', 0.3)
if Print:
    print(diagonal)

evolved = wfn.time_evolve(time, diagonal)
if Print:
    evolved.print_wfn()

Exact evolution of dense quadratic hamiltonians is supported.  Here is an evolution example using a spin restricted Hamiltonian on a number and spin conserving wavefunction

In [ ]:
norb = 4
h1e = numpy.zeros((norb, norb), dtype=numpy.complex128)
for i in range(norb):
    for j in range(norb):
        h1e[i, j] += (i+j) * 0.02
    h1e[i, i] += i * 2.0

hamil = fqe.get_restricted_hamiltonian((h1e,))
wfn = fqe.Wavefunction([[4, 0, norb]])
wfn.set_wfn(strategy='random')
initial_energy = wfn.expectationValue(hamil)
print('Initial Energy: {}'.format(initial_energy))
evolved = wfn.time_evolve(time, hamil)
final_energy = evolved.expectationValue(hamil)
print('Final Energy:   {}'.format(final_energy))

The GSO Hamiltonian is for evolution of quadratic hamiltonians that are spin broken and number conserving.

In [ ]:
norb = 4
h1e = numpy.zeros((2*norb, 2*norb), dtype=numpy.complex128)
for i in range(2*norb):
    for j in range(2*norb):
        h1e[i, j] += (i+j) * 0.02
    h1e[i, i] += i * 2.0

hamil = fqe.get_gso_hamiltonian((h1e,))
wfn = fqe.get_number_conserving_wavefunction(4, norb)
wfn.set_wfn(strategy='random')
initial_energy = wfn.expectationValue(hamil)
print('Initial Energy: {}'.format(initial_energy))
evolved = wfn.time_evolve(time, hamil)
final_energy = evolved.expectationValue(hamil)
print('Final Energy:   {}'.format(final_energy))

The BCS hamiltonian evovles spin conserved and number broken wavefunctions.

In [ ]:
norb = 4
time = 0.001
wfn_spin = fqe.get_spin_conserving_wavefunction(2, norb)
hamil = FermionOperator('', 6.0)
for i in range(0, 2*norb, 2):
    for j in range(0, 2*norb, 2):
        opstring = str(i) + ' ' + str(j + 1)
        hamil += FermionOperator(opstring, (i+1 + j*2)*0.1 - (i+1 + 2*(j + 1))*0.1j)
        opstring = str(i) + '^ ' + str(j + 1) + '^ '
        hamil += FermionOperator(opstring, (i+1 + j)*0.1 + (i+1 + j)*0.1j)
h_noncon = (hamil + hermitian_conjugated(hamil))/2.0
if Print:
    print(h_noncon)

wfn_spin.set_wfn(strategy='random')
if Print:
    wfn_spin.print_wfn()

spin_evolved = wfn_spin.time_evolve(time, h_noncon)
if Print:
    spin_evolved.print_wfn()

Exact Evolution Implementation of Diagonal Coulomb terms

In [ ]:
norb = 4
wfn = fqe.Wavefunction([[5, 1, norb]])
vij = numpy.zeros((norb, norb, norb, norb), dtype=numpy.complex128)
for i in range(norb):
            for j in range(norb):
                vij[i, j] += 4*(i % norb + 1)*(j % norb + 1)*0.21

wfn.set_wfn(strategy='random')

if Print:
    wfn.print_wfn()

hamil = fqe.get_diagonalcoulomb_hamiltonian(vij)

evolved = wfn.time_evolve(time, hamil)
if Print:
    evolved.print_wfn()

Exact evolution of individual n-body anti-Hermitian gnerators

In [ ]:
norb = 3
nele = 4
ops = FermionOperator('5^ 1^ 2 0', 3.0 - 1.j)
ops += FermionOperator('0^ 2^ 1 5', 3.0 + 1.j)
wfn = fqe.get_number_conserving_wavefunction(nele, norb)
wfn.set_wfn(strategy='random')
wfn.normalize()
if Print:
    wfn.print_wfn()
evolved = wfn.time_evolve(time, ops)
if Print:
    evolved.print_wfn()


 Approximate evolution of sums of n-body generators

Approximate evolution can be done for dense operators.

In [ ]:
lih_evolved = lihwfn.apply_generated_unitary(time, 'taylor', lih_hamiltonian, accuracy=1.e-8)
if Print:
    lih_evolved.print_wfn()

In [ ]:
norb = 2
nalpha = 1
nbeta = 1
nele = nalpha + nbeta
time = 0.05
h1e = numpy.zeros((norb*2, norb*2), dtype=numpy.complex128)
for i in range(2*norb):
    for j in range(2*norb):
        h1e[i, j] += (i+j) * 0.02
    h1e[i, i] += i * 2.0
hamil = fqe.get_general_hamiltonian((h1e,))
spec_lim = [-1.13199078e-03, 6.12720338e+00]
wfn = fqe.Wavefunction([[nele, nalpha - nbeta, norb]])
wfn.set_wfn(strategy='random')
if Print:
    wfn.print_wfn()
evol_wfn = wfn.apply_generated_unitary(time, 'chebyshev', hamil, spec_lim=spec_lim)
if Print:
    evol_wfn.print_wfn()

API for determining desired expectation values

In [ ]:
rdm1 = lihwfn.expectationValue('i^ j')
if Print:
    print(rdm1)
val = lihwfn.expectationValue('5^ 3')
if Print:
    print(2.*val)
trdm1 = fqe.expectationValue(lih_evolved, 'i j^', lihwfn)
if Print:
    print(trdm1)
val = fqe.expectationValue(lih_evolved, '5 3^', lihwfn)
if Print:
    print(2*val)

2.B.1 RDMs
In addition to the above API higher order density matrices in addition to hole densities can be calculated.

In [ ]:
rdm2 = lihwfn.expectationValue('i^ j k l^')
if Print:
    print(rdm2)
rdm2 = fqe.expectationValue(lihwfn, 'i^ j^ k l', lihwfn)
if Print:
    print(rdm2)

2.B.2 Hamiltonian expectations (or any expectation values)

In [ ]:
li_h_energy = lihwfn.expectationValue(lih_hamiltonian)
if Print:
    print(li_h_energy)
li_h_energy = fqe.expectationValue(lihwfn, lih_hamiltonian, lihwfn)
if Print:
    print(li_h_energy)

2.B.3 Symmetry operations

In [ ]:
op = fqe.get_s2_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_sz_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_time_reversal_operator()
print(lihwfn.expectationValue(op))
op = fqe.get_number_operator()
print(lihwfn.expectationValue(op))
